<a href="https://colab.research.google.com/github/PaulinaWalasiewicz/Colab-notebooks/blob/main/Copy_of_NN_model_training_with_tabular_data_Vistula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import niezbednych bibliotek
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
# Przygotowanie przykładowych danych
data = {
    'Category': ['Electronics', 'Furniture', 'Clothing', 'Electronics', 'Clothing'],
    'Region': ['North', 'South', 'East', 'West', 'North'],
    'Sales': [200, 300, 150, 400, 250],
    'Time on Platform': [30, 45, 20, 50, 35],
    'Target': [1, 0, 1, 0, 1]
}

# Konwersja danych do pandas DataFrame
df = pd.DataFrame(data)

In [3]:
# Funkcja do kodowania wartości kategorycznych
def encode_categorical_columns(df, columns):
    label_encoders = {}
    for col in columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le
    return label_encoders

# Funkcja do standaryzacji wartości numerycznych
def standardize_columns(df, columns):
    scaler = StandardScaler()
    df[columns] = scaler.fit_transform(df[columns])
    return scaler

# Kodowanie kolumn kategorycznych
categorical_columns = ['Category', 'Region']
label_encoders = encode_categorical_columns(df, categorical_columns)

# Standaryzacja kolumn numerycznych
numerical_columns = ['Sales', 'Time on Platform']
scaler = standardize_columns(df, numerical_columns)

# Podział danych na zbiory treningowy i testowy
features = ['Category', 'Region', 'Sales', 'Time on Platform']
X = df[features]
y = df['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Definicja modelu embeddingów
class EmbeddingModel(nn.Module):
    def __init__(self, embedding_sizes, num_numerical_features):
        """
        Inicjalizacja modelu.
        embedding_sizes: lista krotek (liczba kategorii, rozmiar embeddingow) dla zmiennych kategorycznych.
        num_numerical_features: liczba cech numerycznych.
        """
        super().__init__()

        # Tworzenie listy warstw embeddingow dla zmiennych kategorycznych
        self.embeddings = nn.ModuleList([
            nn.Embedding(num_embeddings=categories, embedding_dim=size)
            for categories, size in embedding_sizes
        ])

        # Warstwy w pełni połączone (fully connected)
        input_size = sum([size for _, size in embedding_sizes]) + num_numerical_features
        self.fc1 = nn.Linear(input_size, 32)  # Pierwsza warstwa w pełni połączona
        self.fc2 = nn.Linear(32, 32)         # Druga warstwa w pełni połączona
        self.fc3 = nn.Linear(32, 16)         # Druga warstwa w pełni połączona
        self.output = nn.Linear(16, 1)      # Warstwa wyjściowa
        self.dropout = nn.Dropout(0.2)      # Dropout dla regularyzacji

    def forward(self, x_categorical, x_numerical):
        """
        Propagacja w przód (forward pass).
        x_categorical: tensor wejściowy dla zmiennych kategorycznych.
        x_numerical: tensor wejściowy dla cech numerycznych.
        """
        # Embeddingi dla zmiennych kategorycznych
        embedded = [emb(x_categorical[:, i]) for i, emb in enumerate(self.embeddings)]
        embedded = torch.cat(embedded, dim=1)  # Konkatenacja embeddingow

        # Połączenie embeddingow ze zmiennymi numerycznymi
        x = torch.cat([embedded, x_numerical], dim=1)

        # Przepływ przez warstwy w pełni połączone z funkcją aktywacji ReLU
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.dropout(torch.relu(self.fc3(x)))

        # Warstwa wyjściowa z funkcją aktywacji sigmoid
        logits = torch.sigmoid(self.output(x))
        return logits,x

In [20]:
# Określenie rozmiarów wektorów embeddingów dla zmiennych kategorycznych
# embedding_sizes: lista krotek (liczba unikalnych wartości w kolumnie, rozmiar embeddingu)
embedding_sizes = [
    (df['Category'].nunique(), 4),  # 'Product Category': liczba kategorii i rozmiar embeddingu
    (df['Region'].nunique(), 4)   # 'Customer Region': liczba regionów i rozmiar embeddingu
]

# Konwersja danych na tensory
# Dane kategoryczne (train)
X_categorical_train = torch.tensor(
    X_train[['Category', 'Region']].values, dtype=torch.long
)

# Dane numeryczne (train)
X_numerical_train = torch.tensor(
    X_train[['Sales', 'Time on Platform']].values, dtype=torch.float
)

# Target (wartości wyjściowe) jako tensor (przekształcenie na kolumnowy tensor)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float).view(-1, 1)

# Utworzenie instancji modelu oraz optymalizatora gradientowego
# EmbeddingModel: model uwzględniający embeddingi i dane numeryczne
model = EmbeddingModel(embedding_sizes, num_numerical_features=2)

# Kryterium strat: funkcja Binary Cross-Entropy (BCELoss)
criterion = nn.BCELoss()

# Optymalizator: Adam z ustaloną wartością learning rate
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Pętla trenująca model
epochs = 50  # Liczba epok
for epoch in range(epochs):
    optimizer.zero_grad()                  # Zerowanie gradientów
    outputs,_ = model(X_categorical_train, X_numerical_train)  # Przepływ danych przez model
    loss = criterion(outputs, y_train_tensor)  # Obliczanie strat
    loss.backward()                     # Wyznaczanie gradientów
    optimizer.step()                    # Aktualizacja wag modelu

# Wyświetlenie oceny modelu w trybie ewaluacji
print(model.eval())

EmbeddingModel(
  (embeddings): ModuleList(
    (0): Embedding(3, 4)
    (1): Embedding(4, 4)
  )
  (fc1): Linear(in_features=10, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [21]:
path = "/content/model/trained_model_2.pth"
# path = "/content/trained_model.pth"
torch.save(model, path)

In [22]:
checkpoint = torch.load(path)
checkpoint.eval()

<ipython-input-22-518e4102219a>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)


EmbeddingModel(
  (embeddings): ModuleList(
    (0): Embedding(3, 4)
    (1): Embedding(4, 4)
  )
  (fc1): Linear(in_features=10, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [23]:
# Przygotowanie danych do inferencji modelu
# ZAD1. Wykonajcie inferencje i zapiszcie wynik dla 10 roznych wartosci data_inf
data_inf = {
    'Category': ['Electronics', 'Furniture', 'Clothing', 'Electronics', 'Clothing', 'Electronics', 'Furniture', 'Clothing', 'Electronics', 'Clothing'],
    'Region': ['North', 'South', 'East', 'West', 'North', 'North', 'South', 'East', 'West', 'North'],
    'Sales': [200, 300, 150, 400, 250, 200, 300, 150, 400, 250],
    'Time on Platform': [30, 45, 20, 50, 35, 30, 45, 20, 50, 35]
}

# Konwersja do pandas DataFrame
df_inf = pd.DataFrame(data_inf)

# Kodowanie kolumn kategorycznych
label_encoders = encode_categorical_columns(df_inf, categorical_columns)

# Standaryzacja kolumn numerycznych
scaler = standardize_columns(df_inf, numerical_columns)

# Podział danych na zbiory treningowy i testowy
features = ['Category', 'Region', 'Sales', 'Time on Platform']
X = df_inf[features]

X_categorical_inf = torch.tensor(
    X[['Category', 'Region']].values, dtype=torch.long
)

# Dane numeryczne (train)
X_numerical_inf = torch.tensor(
    X[['Sales', 'Time on Platform']].values, dtype=torch.float
)

output_inf = checkpoint(X_categorical_inf, X_numerical_inf)

# Wynik inferencji to tzw. logit, im blizej, 0 tym wieksza pewnosc modelu ze Target = 0, im blizej 1, wym wieksza pewnosc modelu ze target = 1. Wszystko pomiedzy 0 i 1 obarczone jest niepewnoscia modelu co do poprawnego wyniku, przy czym najwieksza niepewnosc modelu oznacza wynik inferencji - 0.5
print(output_inf)

(tensor([[1.0000e+00],
        [3.3526e-03],
        [1.0000e+00],
        [3.9072e-11],
        [1.0000e+00],
        [1.0000e+00],
        [3.3526e-03],
        [1.0000e+00],
        [3.9072e-11],
        [1.0000e+00]], grad_fn=<SigmoidBackward0>), tensor([[29.7647,  0.0000,  0.0000, 26.1190,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000, 22.9571,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0533,  3.7829,  0.0000,  0.2429,  3.5930,  0.0000,  0.0000,  3.4123,
          0.0000,  0.0000,  3.6712,  0.0000,  0.0000,  0.0000,  3.9063,  1.9202],
        [28.7151,  0.0000,  0.0000, 24.8327,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000, 21.9016,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, 15.2049,  0.0000,  0.0000, 15.0602,  0.0000,  0.0000, 13.8681,
          0.0000,  0.0000, 15.3042,  0.0000,  0.0000,  0.0000, 16.4147,  8.0222],
        [30.7005,  0.0000,  0.0000, 26.7840,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000

In [ ]:
# ZAD2. Zmodyfikujcie kod tak, aby oprocz wyniku (logit) zostal rowniez zwracany embedding dla rekordu na wyjciu warstwy fc2. Wskazowka: Zmian nalezy dokonac tylko w czesci # Zdefiniowanie modelu
# ZAD3. W definicji modelu, dodajcie kolejna, trzecia warstwe liniowa w pelni polaczona fc3, tak aby fc2 miala wymiar (32, 32) oraz fc3 miala wymiar (32,16)
# ZAD4. Poeksperymentujcie jak wplynie na dokladnosc przewidywania modelu zmiana nastepujacych parametrow modelu i trenera:
#   - dodanie kolejnych warstw liniowych do modelu fc4 i fc5
#   - zmiana wymiaru wejsie-wyjscie warstw liniowych fc1, fc2, fc3, ...
#   - zmniejszenie lub zwiększenie (max lr = 1) wartosci kroku w optymalizatorze (lr)

In [24]:
class ExtendedEmbeddingModel(nn.Module):
    def __init__(self, embedding_sizes, num_numerical_features):
        super().__init__()
        self.embeddings = nn.ModuleList([
            nn.Embedding(num_embeddings=categories, embedding_dim=size)
            for categories, size in embedding_sizes
        ])
        input_size = sum([size for _, size in embedding_sizes]) + num_numerical_features
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)  # New Layer
        self.fc5 = nn.Linear(8, 4)  # New Layer
        self.output = nn.Linear(4, 1)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x_categorical, x_numerical):
        embedded = [emb(x_categorical[:, i]) for i, emb in enumerate(self.embeddings)]
        embedded = torch.cat(embedded, dim=1)
        x = torch.cat([embedded, x_numerical], dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))  # Pass through fc4
        x = torch.relu(self.fc5(x))  # Pass through fc5
        logits = torch.sigmoid(self.output(x))
        return logits


In [28]:
model = ExtendedEmbeddingModel(embedding_sizes, num_numerical_features=2)

# Kryterium strat: funkcja Binary Cross-Entropy (BCELoss)
criterion = nn.BCELoss()

# Optymalizator: Adam z ustaloną wartością learning rate
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Pętla trenująca model
epochs = 50  # Liczba epok
for epoch in range(epochs):
    optimizer.zero_grad()                  # Zerowanie gradientów
    outputs= model(X_categorical_train, X_numerical_train)  # Przepływ danych przez model
    loss = criterion(outputs, y_train_tensor)  # Obliczanie strat
    loss.backward()                     # Wyznaczanie gradientów
    optimizer.step()                    # Aktualizacja wag modelu

# Wyświetlenie oceny modelu w trybie ewaluacji
print(model.eval())

ExtendedEmbeddingModel(
  (embeddings): ModuleList(
    (0): Embedding(3, 4)
    (1): Embedding(4, 4)
  )
  (fc1): Linear(in_features=10, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=4, bias=True)
  (output): Linear(in_features=4, out_features=1, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)


In [29]:
path = "/content/model/trained_model_3.pth"
# path = "/content/trained_model.pth"
torch.save(model, path)

In [31]:
checkpoint2 = torch.load(path)
checkpoint2.eval()
output_inf = checkpoint2(X_categorical_inf, X_numerical_inf)

# Wynik inferencji to tzw. logit, im blizej, 0 tym wieksza pewnosc modelu ze Target = 0, im blizej 1, wym wieksza pewnosc modelu ze target = 1. Wszystko pomiedzy 0 i 1 obarczone jest niepewnoscia modelu co do poprawnego wyniku, przy czym najwieksza niepewnosc modelu oznacza wynik inferencji - 0.5
print(output_inf)

tensor([[1.0000e+00],
        [5.4603e-01],
        [1.0000e+00],
        [5.8786e-18],
        [1.0000e+00],
        [1.0000e+00],
        [5.4603e-01],
        [1.0000e+00],
        [5.8786e-18],
        [1.0000e+00]], grad_fn=<SigmoidBackward0>)


<ipython-input-31-d051542a283c>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint2 = torch.load(path)


In [32]:
class ModifiedDimensionsEmbeddingModel(nn.Module):
    def __init__(self, embedding_sizes, num_numerical_features):
        super().__init__()
        self.embeddings = nn.ModuleList([
            nn.Embedding(num_embeddings=categories, embedding_dim=size)
            for categories, size in embedding_sizes
        ])
        input_size = sum([size for _, size in embedding_sizes]) + num_numerical_features
        self.fc1 = nn.Linear(input_size, 128)  # Increased size
        self.fc2 = nn.Linear(128, 64)         # Increased size
        self.fc3 = nn.Linear(64, 32)          # Increased size
        self.output = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x_categorical, x_numerical):
        embedded = [emb(x_categorical[:, i]) for i, emb in enumerate(self.embeddings)]
        embedded = torch.cat(embedded, dim=1)
        x = torch.cat([embedded, x_numerical], dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.dropout(torch.relu(self.fc3(x)))
        logits = torch.sigmoid(self.output(x))
        return logits


In [33]:
model = ModifiedDimensionsEmbeddingModel(embedding_sizes, num_numerical_features=2)

# Kryterium strat: funkcja Binary Cross-Entropy (BCELoss)
criterion = nn.BCELoss()

# Optymalizator: Adam z ustaloną wartością learning rate
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Pętla trenująca model
epochs = 50  # Liczba epok
for epoch in range(epochs):
    optimizer.zero_grad()                  # Zerowanie gradientów
    outputs= model(X_categorical_train, X_numerical_train)  # Przepływ danych przez model
    loss = criterion(outputs, y_train_tensor)  # Obliczanie strat
    loss.backward()                     # Wyznaczanie gradientów
    optimizer.step()                    # Aktualizacja wag modelu

# Wyświetlenie oceny modelu w trybie ewaluacji
print(model.eval())

path = "/content/model/trained_model_4.pth"
# path = "/content/trained_model.pth"
torch.save(model, path)

checkpoint3 = torch.load(path)
checkpoint3.eval()
output_inf = checkpoint3(X_categorical_inf, X_numerical_inf)

# Wynik inferencji to tzw. logit, im blizej, 0 tym wieksza pewnosc modelu ze Target = 0, im blizej 1, wym wieksza pewnosc modelu ze target = 1. Wszystko pomiedzy 0 i 1 obarczone jest niepewnoscia modelu co do poprawnego wyniku, przy czym najwieksza niepewnosc modelu oznacza wynik inferencji - 0.5
print(output_inf)

ModifiedDimensionsEmbeddingModel(
  (embeddings): ModuleList(
    (0): Embedding(3, 4)
    (1): Embedding(4, 4)
  )
  (fc1): Linear(in_features=10, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=1, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)
tensor([[1.0000e+00],
        [1.0000e+00],
        [1.0000e+00],
        [1.0701e-19],
        [1.0000e+00],
        [1.0000e+00],
        [1.0000e+00],
        [1.0000e+00],
        [1.0701e-19],
        [1.0000e+00]], grad_fn=<SigmoidBackward0>)


<ipython-input-33-81fbd2ce83ee>:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint3 = torch.load(path)


In [36]:
# Use the default EmbeddingModel
learning_rates = [0.001, 0.01, 0.1, 0.5, 1.0]  # Varying learning rates

results = {}
for lr in learning_rates:
    model = EmbeddingModel(embedding_sizes, num_numerical_features=2)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        optimizer.zero_grad()
        logits, _ = model(X_categorical_train, X_numerical_train)  # Unpack the tuple
        loss = criterion(logits, y_train_tensor)
        loss.backward()
        optimizer.step()

    # Evaluate performance and store results
    model.eval()
    with torch.no_grad():
        y_pred, _ = model(X_categorical_train, X_numerical_train)  # Unpack here as well
        y_pred_binary = (y_pred > 0.5).float()
        accuracy = (y_pred_binary == y_train_tensor).float().mean().item()
        results[lr] = accuracy

# Print results
print("Learning Rate Results:")
for lr, acc in results.items():
    print(f"Learning Rate: {lr}, Accuracy: {acc:.4f}")

Learning Rate Results:
Learning Rate: 0.001, Accuracy: 1.0000
Learning Rate: 0.01, Accuracy: 1.0000
Learning Rate: 0.1, Accuracy: 1.0000
Learning Rate: 0.5, Accuracy: 0.7500
Learning Rate: 1.0, Accuracy: 0.7500
